### INF555 - Constraint-based Modeling and Algorithms for Decision Making

### Ecole Polytechnique - Master of Artificial Intelligence & Advanced Visual Computing (MAI)

# TP2: Boolean SAT Models and SAT Solvers 
In today's TP, 
* we will first model graph coloring problems as Boolean satisfaction problems in MiniZinc usinf Chuffed's SAT solver and do the same for the N-Queens problem;
* then we will a simple Python program for computing one step of **Davis-Putnam** elimination and some **Boolean Constraint Propagation(BCP)**.

# Part 1: Boolean modeling

## Graph Coloring
Let us start with your already favorite example: coloring the Australiant map.

In [207]:
from IPython.core import page
page.page = print

In [208]:
import inf555

In [209]:
!vimcat.sh aust_param.dzn

Vertices = {wa, nt, sa, q, nsw, v};
NbEdges = 1..9;
Edges1 = [wa, wa, nt, nt, sa, sa, sa, q, nsw];
Edges2 = [nt, sa, sa, q, q, nsw, v, nsw, v];


As a reminder, we have already solved the decision problem to coloring a graph defined in the format above with at most `nc` colors, using the following MiniZinc model with integer decision variables:

In [210]:
!vimcat.sh aust_param.mzn

int: nc;
enum Vertices;
set of int: NbEdges;
array[NbEdges] of Vertices: Edges1;
array[NbEdges] of Vertices: Edges2;

array[Vertices] of var int: Color;
constraint forall (color in Color)(color >=0 /\ color < nc);
constraint forall (n in NbEdges)(Color[Edges1[n]] != Color[Edges2[n]]);

solve satisfy;


In [211]:
print(inf555.minizinc('aust_param.mzn', 'aust_param.dzn', data={'nc': 3}, solver=inf555.chuffed))

Solution(Color=[2, 1, 0, 2, 1, 2], _checker='')


## Question 1. Write a BOOLEAN model for the graph coloring problem 
* in a [bool_graph.mzn](bool_graph.mzn) file
* to check the coloriability with `nc` colors of the graph given in [aust_param.dzn](aust_param.dzn) file.

One will use only
* Boolean (`bool`) variables
* and put the constraints in CNF, i.e., 
 * conjunctions (`/\` or `forall`)
 * of disjunctions (`\/` or `exists`)

We use the Chuffed solver that relies on a SAT solver for the Boolean part.

In [212]:
print(inf555.minizinc('bool_graph.mzn', 'aust_param.dzn', data={'nc': 3}, solver=inf555.chuffed))

Solution(Color=[[True, False, False], [False, True, False], [False, False, True], [True, False, False], [False, True, False], [True, False, False]], _checker='')


## N-Queens 

Last week we saw a constraint-based model over the integer domain for solving the Nqueens problem.

In [213]:
!vimcat.sh nqueens.mzn

int: n;
array[1..n] of var 1..n: queens;

constraint forall (i, j in 1..n where i < j) (
   queens[i] != queens[j] /\
   queens[i] != queens[j] + j - i /\
   queens[i] != queens[j] + i - j
);

solve satisfy;

output [if fix(queens[j]) == i then "Q" else "." endif ++
        if j == n then "\n" else " " endif | i,j in 1..n]


In [214]:
print(inf555.minizinc('nqueens.mzn', data={'n': 16}, solver=inf555.chuffed))

. . . . . . Q . . . . . . . . .
. . . . . . . . Q . . . . . . .
. . . . . Q . . . . . . . . . .
. . . . . . . . . . . Q . . . .
. . . . . . . . . Q . . . . . .
Q . . . . . . . . . . . . . . .
. . . . . . . . . . Q . . . . .
. Q . . . . . . . . . . . . . .
. . . . Q . . . . . . . . . . .
. . Q . . . . . . . . . . . . .
. . . . . . . Q . . . . . . . .
. . . . . . . . . . . . . Q . .
. . . Q . . . . . . . . . . . .
. . . . . . . . . . . . . . Q .
. . . . . . . . . . . . Q . . .
. . . . . . . . . . . . . . . Q



## Question 2. Write a BOOLEAN model for the N-Queens problem
* in MiniZinc in a [bool_nqueens.mzn](bool_nqueens.mzn) file
* using Boolean variables only 
* and test it on $n=16$

In [215]:
!vimcat.sh bool_nqueens.mzn

int: n;
array[1..n,1..n] of var bool: queens;

constraint 
    forall (i in 1..n) 
    (
        exists(j in 1..n) (queens[i,j])
    );

constraint 
    forall (i in 1..n) 
    (
        exists(j in 1..n) (queens[j,i])
    );

constraint 
    forall (i in 1..n) 
    (
        forall(j,k in 1..n where j!=k) 
            (not (queens[i,j] /\ queens[i,k]))
    );

constraint 
    forall (i in 1..n) 
    (
        forall(j,k in 1..n where j!=k) 
            (not (queens[j,i] /\ queens[k,i]))
    );

constraint 
    forall (i,j in 1..n) 
    (
        forall(k in 1..n) 
            ((not (queens[i,j] /\ queens[i+k,j+k])) /\ (not (queens[i,j] /\ queens[i-k,j+k])))
    );

solve satisfy;

output [if fix(queens[i,j]) then "Q" else "." endif ++
        if j == n then "\n" else " " endif | i,j in 1..n]


In [216]:
print(inf555.minizinc('bool_nqueens.mzn', data={'n': 16}, solver=inf555.chuffed))

. . . . . . . . . Q . . . . . .
. . . . . . . Q . . . . . . . .
. . . . . . . . . . Q . . . . .
. . . Q . . . . . . . . . . . .
. . . . . . Q . . . . . . . . .
. . . . . . . . . . . . . . . Q
. . . . . Q . . . . . . . . . .
. . . . . . . . . . . . . . Q .
. . . . . . . . . . . Q . . . .
. . . . . . . . . . . . . Q . .
. . . . . . . . Q . . . . . . .
. . . . . . . . . . . . Q . . .
. Q . . . . . . . . . . . . . .
. . . . Q . . . . . . . . . . .
. . Q . . . . . . . . . . . . .
Q . . . . . . . . . . . . . . .



# Part 2: SAT Solver

Let us start with a small Python function that reduces clauses _à la_ Davis-Putnam.

You might want to re-read https://docs.python.org/3/tutorial/index.html chapters 3, 4 and 5 if you are not familiar with Python.

We will represent
* clauses by sets,
* variables by positive integers,
* and negation by negative values.

For instance the clause _(b + c' + f')_ becomes `{2, -3, -6}`

We will use the classical encoding of CNF files where
* the first line of the file is `p cnf V C`
 * where `V` is the number of variables
 * and `C` the number of clauses
* then each clause is written on a line,
 * with literals as above,
 * separated by spaces,
 * line ended by a conventional `0`.

## Question 3. Modify the file named [dp.py](dp.py)([VSCode](http://localhost:8080/?folder=/home/jovyan)) 
such that the function `davis_putnam`
* takes a list of clauses (sets) and a variable (integer),
* returns a new list of sets with the given variable eliminated***

We should have, as shown in the slides:

`davis_putnam([{1, 2, 3}, {2, -3, -6}, {-2, 5}], 2)` -> `[{1, 3, 5}, {-6, -3, 5}]`

For the moment, we do not care about the satisfiability of the resulting clauses.

In [217]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [218]:
from dp import davis_putnam

In [219]:
%pycat dp.py

# These are Python3 type-annotations
from typing import List, Set
Var = int
Literal = int
Clause = Set[Literal]
Instance = List[Clause]


def davis_putnam(instance: Instance, var: Var) -> Instance:
    """Eliminate var from instance using Davis-Putnam algorithm."""
    L_no_x = []
    L_xp = [] 
    L_xn = []
    for c in instance:
        if var in c:
            L_xp.append(c - {var})
        elif -var in c:
            L_xn.append(c - {-var})
        else:
            L_no_x.append(c)
    instance = []
    for c in L_no_x:
        instance.append(c)
    
    for cp in L_xp:
        for cn in L_xn:
            instance.append(cp | cn)


    return instance



In [220]:
davis_putnam([{1, 2, 3}, {2, -3, -6}, {-2, 5}], 2)

[{1, 3, 5}, {-6, -3, 5}]

Ensure that the result is correct even if the given variable appears only once or even not at all.

In [221]:
davis_putnam([{1, 2, 3}, {2, -3, -6}, {-2, 5}], 5)

[{1, 2, 3}, {-6, -3, 2}]

In [222]:
davis_putnam([{1, 2, 3}, {2, -3, -6}, {-2, 5}], 4)

[{1, 2, 3}, {-6, -3, 2}, {-2, 5}]

## Question 4. What result indicates a SAT problem? an UNSAT problem?

_Give examples in the cells below and write your answer here_

make sure there each variable exists at most once in a clause

run davis_putnam function until only one clause remian

if the last claue is not empty then SAT else UNSAT

SAT Example

In [223]:
davis_putnam([{1, 2, 3}, {2, -3, -6}, {-2, 5}], 3)

[{-2, 5}, {-6, 1, 2}]

In [224]:
davis_putnam([{-2, 5}, {-6, 1, 2}], 2)

[{-6, 1, 5}]

UNSAT Example

In [225]:
davis_putnam([{1, 2}, {2}, {-2, -5},{5}], 1)

[{2}, {-5, -2}, {5}]

In [226]:
davis_putnam([{2}, {-2, -5}, {5}], 5)

[{2}, {-2}]

In [227]:
davis_putnam([{2}, {-2}], 2)

[set()]

## Question 5. Give one example where the number of clauses grows

_give comments here_

for a list of clauses, we replace all clauses containing x or -x with clauses in number of at lost n*m 
(where n in the number of clauses containing x and m is the number of clauses containing -x, at most because some clause maybe a unit clause containg only x or -x)

So when n*m > n+m, the number of clauses grows

in this case n=m=3, and no unit clause, then new list has n*m=9 clauses and origin list has only 6 clauses


In [228]:
davis_putnam([{1, 2}, {1,3}, {1,4},{-1,5},{-1,6},{-1,7}], 1)

[{2, 5}, {2, 6}, {2, 7}, {3, 5}, {3, 6}, {3, 7}, {4, 5}, {4, 6}, {4, 7}]

Let us now implement the part that is common to DP and D(P)LL, i.e., the unit propagation or BCP.

## Question 6. In the file [bcp.py](bcp.py), define a function `unit_propagate` 
* that takes an instance and a current assignment, as an ordered list of 'Unassigned', 'True', 'False' values, 
* and applies all Boolean clause rewritings
* before returning the resulting instance and assignment.

It should have done all possible propagations but can be naive and inefficient.

* An empty instance with a non-empty assignment will indicate **SAT**
* whereas by convention an empty assignment will indicate **UNSAT**.

In [229]:
%pycat bcp.py

from enum import Enum
from typing import List, Set
Var = int
Literal = int
Clause = Set[Literal]
Instance = List[Clause]

Value = Enum('Value', ['Unassigned', 'True', 'False'])
Assignment = List[Value]


def unit_propagate(instance: Instance, assignment: Assignment) -> (Instance, Assignment):
    """Apply BCP to current instance and update assignment."""
    l_true = []
    l_false = []
    res = []
    for v in range(len(assignment)):
        if assignment[v] == "True":
            l_true.append(v+1)
        elif assignment[v] == "False":
            l_false.append(v+1)
        v += 1
    for c in instance:
        f_in = True
        lc = c.copy()
        for vp in l_true:
            if vp in c:
                f_in = False
            lc = lc - {vp, -vp}
        for vn in l_false:
            if -vn in c:
                f_in = False
            lc = lc - {vn, -vn}
        if f_in:
            res.append(lc)
    instance = res.copy()
    res = []
    l_unit = []
    for c in instan

In [230]:
from bcp import unit_propagate

In [231]:
unit_propagate([{1, 2, 3}, {2, -3, -6}, {-2, 5}, {-2, -5, 4}], ['Unassigned', 'True', 'Unassigned', 'Unassigned', 'Unassigned', 'Unassigned'])

([{5}, {-5, 4}],
 ['Unassigned', 'True', 'Unassigned', 'Unassigned', 'True', 'Unassigned'])

In [232]:
unit_propagate([{5}, {-5, 4}], ['Unassigned', 'True', 'Unassigned', 'Unassigned', 'True', 'Unassigned'])

([{4}], ['Unassigned', 'True', 'Unassigned', 'True', 'True', 'Unassigned'])

In [233]:
unit_propagate([{4}], ['Unassigned', 'True', 'Unassigned', 'True', 'True', 'Unassigned'])

([], ['Unassigned', 'True', 'Unassigned', 'True', 'True', 'Unassigned'])

## Question 7. Complete your program to make it a complete SAT solver

In [234]:
def SAT_or_not(instance):
    if len(instance) == 2:
        if (len(list(instance[0])) == 1) and (len(list(instance[1])) == 1):
            if (list(instance[0]))[0] == -(list(instance[1]))[0]:
                return 1
    if len(instance) == 0:
        return 0
    return -1

In [235]:
def SAT_solver(instance, assignment):
    times = 0
    times_max = 10000
    
    while (times<times_max):
        new_instance,new_assignment = unit_propagate(instance, assignment)
        while (new_instance != instance or new_assignment != assignment):
            instance = new_instance
            assignment = new_assignment
            new_instance,new_assignment = unit_propagate(instance, assignment)
            #print(instance,assignment)
            
        res = SAT_or_not(instance)
        if res == 1:
            return 'UNSAT'
        if res == 0:
            return 'SAT', assignment

        var = 0
        for i in range(len(instance)):
            for j in range(i+1,len(instance)):
                ci = list(instance[i])
                cj = list(instance[j])
                for x in ci:
                    for y in cj:
                        if x==y or x==-y:
                            var = abs(x)
                            break
        if var == 0:
            return 'var error',instance,assignment
        instance = davis_putnam(instance, var)
        #print(var)
        #print(instance)
        
        if len(instance) == 1:
            for v in list(instance[0]):
                if v>0:
                    assignment[v-1] = 'True'
                else:
                    assignment[-v-1] = 'False'
                    
        #print(assignment)       
        
        times += 1
    
    return 'time error',new_instance,new_assignment
                                          

In [236]:
SAT_solver([{1, 2, 3}, {2, -3, -6}, {-2, 5}, {-2, -5, 4}], ['Unassigned', 'True', 'Unassigned', 'Unassigned', 'Unassigned', 'Unassigned'])

('SAT', ['Unassigned', 'True', 'Unassigned', 'True', 'True', 'Unassigned'])

In [241]:
SAT_solver([{1, 2, 3}, {2, -3, -6}, {-2, 5}, {-2, -5, 4}], ['Unassigned', 'False', 'Unassigned', 'Unassigned', 'Unassigned', 'Unassigned'])

('SAT', ['True', 'False', 'Unassigned', 'Unassigned', 'Unassigned', 'False'])

In [238]:
SAT_solver([{1,2},{2}, {-2}], ['Unassigned', 'Unassigned'])

'UNSAT'

## Question 8. Use your SAT solver to solve the 4-queens problem

by generating the query

* either using your favorite editor
* or by writing a small Python program [expand.py](expand.py) for it

This is essentially what the transformation of MiniZinc to FlatZinc does for Boolean MiniZinc models.

## Do not forget to save your files on your machine
[dp.py](dp.py)

[bool_graph.mzn](bool_graph.mzn)

[bool_nqueens.mzn](bool_nqueens.mzn)

[bcp.py](bcp.py)

[expand.py](sat.py) (optionally)

[SAT.ipynb](SAT.ipynb) with the answers

## and send us your work via the Moodle